In [13]:
import xpress as xp
import pandas as pd
import numpy as np
import math

In [14]:
# VERY_LARGE_NUMBER = 10000
lambda1 = 1
lambda2 = 10000*lambda1


model = xp.problem()
model.controls.maxtime = 400 #max search time
df = pd.read_excel("person.xlsx")

In [15]:
#person info 
num_guests = len(df)

In [16]:

matric = df['Matric'].tolist()
for index,value in enumerate(matric):
    if not math.isnan(value):
        matric[index] = int(matric[index]) 

gender = df['Gender'].tolist()
for index,value in enumerate(gender):
    if value == 'M':
        gender[index] = 1
    elif value == "F":
        gender[index] = -1
    else: # None
        gender[index] = 0

In [17]:
ppl_type = df['TYPE'].tolist()
NoA = 0
NoS = 0
NoF = 0
NoP = len(ppl_type)

is_ALUMNI = []
is_STUDENT = []
is_FELLOW = []
for index,value in enumerate(ppl_type):
    if value == 'ALUMNI':
        is_ALUMNI.append(1)
        is_STUDENT.append(0)
        is_FELLOW.append(0)
        NoA += 1
    elif value == 'STUDENT':
        is_ALUMNI.append(0)
        is_STUDENT.append(1)
        is_FELLOW.append(0)
        NoS += 1
    elif value == 'FELLOW':
        is_ALUMNI.append(0)
        is_STUDENT.append(0)
        is_FELLOW.append(1)
        NoF += 1


# high table idx
notes = df['Notes'].tolist()
to_high_table_id = []
for index,value in enumerate(notes):
    if type(value) == str:
        to_high_table_id.append(1)
    else:
        to_high_table_id.append(0)
# table info

num_tables = 6
table_seats = [ 21, 16, 26, 10, 20, 17 ]
high_table_id = [ 1, 0, 0, 0, 0, 0 ]


In [18]:

# NoA constrants for each table
cons_NoA = [NoA/NoP * table_seats[i] +1 for i in range(len(table_seats))]
# NoS constrants for each table
cons_NoS = [NoS/NoP * table_seats[i] +1 for i in range(len(table_seats))]
# NoF constrants for each table
cons_NoF = [NoF/NoP * table_seats[i] +1 for i in range(len(table_seats))]

maxseats = sum(table_seats)

In [19]:
# decision variables

x=np.array([[xp.var(name=f'x{i}_{j}',vartype=xp.binary) for j in range(num_tables)] for i in range(num_guests)],dtype=xp.npvar)
gender_diff = np.array([xp.var(name=f'gender_diff{i}',vartype=xp.binary) for i in range(num_tables)],dtype=xp.npvar)
max_matric = np.array([xp.var(name = f'max_matric{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)
min_matric = np.array([xp.var(name = f'min_matric{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)
# max_matric = [model.addVariable() for i in range(num_tables)]
# min_matric = [model.addVariable() for i in range(num_tables)]

model.addVariable(x)
model.addVariable(gender_diff)
model.addVariable(max_matric)
model.addVariable(min_matric)


In [20]:
# constrants

# one person one seat
for i in range(num_guests):
    model.addConstraint(xp.Sum(x[i, j] for j in range(num_tables)) == 1)

# table capacity
for i in range(num_tables):
       model.addConstraint(xp.Sum(x[j, i] for j in range(num_guests)) == table_seats[i])

# specific diner in specific table
for i in range(len(to_high_table_id)):
    if to_high_table_id[i] == 1:
        for j in range(len(high_table_id)):
            if high_table_id[j] == 1:
                model.addConstraint(x[i,j] == 1)

# alumni, fellows, students
for i in range(len(table_seats)):
    model.addConstraint(xp.Sum(x[j,i] * is_ALUMNI[j] for j in range(num_guests)) <= cons_NoA[i])
    model.addConstraint(xp.Sum(x[j,i] * is_STUDENT[j] for j in range(num_guests)) <= cons_NoS[i])
    model.addConstraint(xp.Sum(x[j,i] * is_FELLOW[j] for j in range(num_guests)) <= cons_NoF[i])

for i in range(NoP):
    if not math.isnan(matric[i]):
        for jj in range(num_tables): 
            model.addConstraint(max_matric[jj] >= x[i,jj]*matric[i] - 10000*(1-x[i,jj]))
            model.addConstraint(min_matric[jj] <= x[i,jj]*matric[i] + 10000 * (1-x[i,jj]))

#gender_diff constraints  abs constraint
for j in range(num_tables):
    model.addConstraint(gender_diff[j] >= xp.Sum(x[i,j] * gender[i] for i in range(num_guests)))
    model.addConstraint(gender_diff[j] >= -xp.Sum(x[i,j] * gender[i] for i in range(num_guests)))


In [21]:
# """test constraints"""
# for i in range(NoP):
#     if not math.isnan(matric[i]):
#         for jj in range(num_tables): 
#             model.addConstraint(max_matric[jj] >= x[i,jj])
# # for j in range(num_tables):
# #     model.addConstraint(max_matric[j] == xp.max(x[:,j]))

# """test obj"""
# sum_max_matric = xp.Sum(max_matric[j] for j in range(num_tables))
# model.setObjective(sum_max_matric)

# objective func

range_matric = xp.Sum(max_matric[j] - min_matric[j] for j in range(num_tables))
sum_gender_diff = xp.Sum(gender_diff[j] for j in range(num_tables))


model.setObjective(lambda1*sum_gender_diff+lambda2*range_matric)
# model.addObjective(range_matric)
# model.addobj(gender_diff, [1.0]*len(gender_diff))

model.optimize()
res = model.getSolution(x)
res_to_save = {}
# convert  res to list(int)
table_person_mapping = res.tolist()
table_person_mapping = [[int(i) for i in j] for j in table_person_mapping]

res_to_save = {
    'table_person_mapping': table_person_mapping,
    'gender':gender,
    'matric':matric,
    'is_ALUMNI':is_ALUMNI,
    'is_STUDENT':is_STUDENT,
    'is_FELLOW':is_FELLOW
}
import json
with open('stage1_res.json','w') as f:
    json.dump(res_to_save,f)

for j in range(num_tables):
    out = ""
    out += f"table {j+1}:\n"
    
    guest_id = [i for i in range(num_guests) if res[i,j] == 1]
    out += f"guests: {[i+1 for i in guest_id]}\n"

    num_male = sum([1 if gender[i] == 1 else 0 for i in guest_id])
    num_female = sum([1 if gender[i] == -1 else 0 for i in guest_id])
    num_unknown_gender = sum([1 if gender[i] == 0 else 0 for i in guest_id])
    out += f"male: {num_male}, female: {num_female}, unknown: {num_unknown_gender}\n"
    num_alumni = sum([1 if is_ALUMNI[i] == 1 else 0 for i in guest_id])
    num_student = sum([1 if is_STUDENT[i] == 1 else 0 for i in guest_id])
    num_fellow = sum([1 if is_FELLOW[i] == 1 else 0 for i in guest_id])
    out += f"alumni: {num_alumni}, student: {num_student}, fellow: {num_fellow}\n"

    matric_info = [matric[i] for i in guest_id]
    out += f"matric info: {matric_info}\n"
    score_gender_diff = abs(num_female-num_male)
    out += f"gender diff: {score_gender_diff}\n"
    score_matric_diff = max(matric_info) - min(matric_info)
    out += f"matric diff: {score_matric_diff}\n"
    out+= '\n'
    print(out)

FICO Xpress v9.2.4, Hyper, solve started 15:51:43, Aug 21, 2024
Heap usage: 956KB (peak 1164KB, 769KB system)
Minimizing MILP noname using up to 8 threads and up to 8006MB memory, with these control settings:
MAXTIME = 400
OUTPUTLOG = 1
Original problem has:
      1465 rows          678 cols         5111 elements       678 entities
Presolved problem has:
      1128 rows          612 cols         4200 elements       612 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 1416KB (peak 2358KB, 769KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.20e+04] / [ 2.44e-04,  1.97e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Symmetric problem: generators: 41, support set: 402
 Number of orbits: 156, largest orbit: 8
 Row orbits: 288, row supp

    1652  1910003.000 -297637046.7     13   1126     63 3.00e+08     192      9
    1753  1910003.000 -286398346.0     13   1221     88 2.88e+08     204      9
    1854  1910003.000 -286398346.0     13   1221     92 2.88e+08     209     10
    1954  1910003.000 -286398346.0     13   1221     52 2.88e+08     275     10
    2060  1910003.000 -286398346.0     13   1243     56 2.88e+08     240     10
Resetting tree to root.

Performing root presolve...

Reduced problem has:    1273 rows     612 columns      8971 elements
Presolve dropped   :       0 rows       0 columns        13 elements
Presolve tightened :        98 elements
Symmetric problem: generators: 41, support set: 402
 Number of orbits: 156, largest orbit: 8
 Row orbits: 288, row support: 744
Will try to keep branch and bound tree memory usage below 129MB
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
416621      -495350000.0      D    131     0        .000000    10
417754      -402401052.2      P      0    

    3427  1580003.000 -286398346.0     36    954     72 2.88e+08     122     28
    3527  1580003.000 -286398346.0     36   1055     64 2.88e+08     154     29
    3629  1580003.000 -286398346.0     36   1095     87 2.88e+08     164     29
    3733  1580003.000 -286398346.0     36   1145    120 2.88e+08      95     30
    3835  1580003.000 -286398346.0     36   1171     68 2.88e+08     186     30
    3948  1580003.000 -286398346.0     36   1296     63 2.88e+08     131     30
    4051  1580003.000 -286398346.0     36   1398     44 2.88e+08     193     31
    4151  1580003.000 -286398346.0     36   1444     31 2.88e+08     166     31
    4252  1580003.000 -286398346.0     36   1595    116 2.88e+08      77     32
Elapsed time (sec): 32, estimated tree completion: 0.00010
Heap usage: 248MB (peak 248MB, 16MB system)
B&B tree size: 8.5MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    4352  1580003.000 -286398346.0     36   1635     59 2.88e+08    

   18140  1580003.000 -274545118.3     36  12321     38 2.76e+08     220     70
   19146  1580003.000 -273549741.5     36  12999     84 2.75e+08     208     72
   20157  1580003.000 -272182467.1     36  13844    136 2.74e+08      32     75
   21166  1580003.000 -271986345.0     36  14545     96 2.74e+08     115     78
   22171  1580003.000 -271986345.0     36  15373     67 2.74e+08      86     81
   23173  1580003.000 -271619873.8     36  16298     28 2.73e+08     240     84
   24173  1580003.000 -266001188.3     36  16993     20 2.68e+08     229     87
   25180  1580003.000 -265025926.2     36  17749     76 2.67e+08     115     89
   26184  1580003.000 -264909240.4     36  18577     24 2.66e+08     250     92
   27198  1580003.000 -252509498.1     36  19287     59 2.54e+08     144     95
   28208  1580003.000 -251047569.6     36  19986     64 2.53e+08     132     98
   29209  1580003.000 -250036649.9     36  20663     36 2.52e+08     188    100
   30210  1580003.000 -249391343.3     3

STOPPING - MAXTIME limit reached (TIMELIMIT=400.00  time=400.00).
STOPPING - MAXTIME limit reached (TIMELIMIT=400.00  time=400.00).
 *** Search unfinished ***    Time:   400.20 Nodes:     158341
Numerical issues encountered:
   Dual failures    :     96 out of    352710 (ratio: 0.0003)
   Singular bases   :     32 out of    281275 (ratio: 0.0001)
Final MIP objective                   : 1.560003000000000e+06
Final MIP bound                       : -1.312060229980190e+08
  Solution time / primaldual integral :    400.20s/ 100.000000%
  Number of solutions found / nodes   :        38 /    158341
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max integer violation     (abs    ) :       0.0
table 1:
guests: [5, 12, 19, 23, 28, 30, 33, 38, 39, 42, 48, 60, 68, 74, 76, 78, 83, 92, 96, 98, 106]
male: 4, female: 5, unknown: 12
alumni: 11, student: 6, fellow: 4
matric info: [1960, 1965, 1954, 1952, 2008, 1964, 1959, 1966, 1966, 1963, 2007, 2013, 2013, 2010, 2012, 2011, 2012, 1988

In [22]:
res_to_save = {}
# convert  res to list(int)
table_person_mapping = res.tolist()
table_person_mapping = [[int(i) for i in j] for j in table_person_mapping]

res_to_save = {
      'table_person_mapping':table_person_mapping,
      'gender':gender,
      'matric':matric,
      'is_ALUMNI':is_ALUMNI,
      'is_STUDENT':is_STUDENT,
      'is_FELLOW':is_FELLOW
}
import json
with open('stage1_res.json','w') as f:
    json.dump(res_to_save,f)
